In [1]:
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt

In [3]:
rootdir = '/raid1/chen423/serdp/archive/GRL2020/'

### major AR-related functions

In [4]:
def save_to_nc(outfile, indata):
    outgroup = nc.Dataset(outfile, 'w', format='NETCDF4')

    # dimension
    xdim = outgroup.createDimension('x', 450)
    ydim = outgroup.createDimension('y', 450)

    # vars
    xvar = outgroup.createVariable('x', 'i4', ('x'))
    xvar[:] = np.arange(450)
    yvar = outgroup.createVariable('y', 'i4', ('y'))
    yvar[:] = np.arange(450)

    var_lat = outgroup.createVariable('lat', 'f4', ('y', 'x'))
    var_lat.long_name = 'Latitude'
    var_lat.standard_name = 'latitude'
    var_lat.units = 'degrees_north'
    var_lon = outgroup.createVariable('lon', 'f4', ('y', 'x'))
    var_lon.long_name = 'Longitude'
    var_lon.standard_name = 'longitude'
    var_lon.units = 'degrees_east'
    var_tag = outgroup.createVariable('dist_to_coast', 'f4', ('y', 'x'))
    var_tag.long_name = 'Distance to coast'
    var_tag.units = 'km'
    var_tag.coordinates = 'lon lat'
    
    var_lat[:] = wrf_lat
    var_lon[:] = wrf_lon
    var_tag[:] = indata
    
    outgroup.script = '/usr1/chen423/serdp/tools/ipython/paperX/AR-SST/step2.compute_land_pt_dist_to_coastal.ipynb'

    outgroup.close()

In [5]:
reffile = rootdir + 'data/common_ref/latlon.nc'
wrf_lat = xr.open_dataset(reffile).XLAT_M.values
wrf_lon = xr.open_dataset(reffile).XLONG_M.values
wrf_landmask = xr.open_dataset(reffile).LANDMASK.values

landmask_lake_fill = wrf_landmask.copy()

landmask_lake_fill[:,190:450] = 1

R=4
for i in np.arange(R, 450-R):
    for j in np.arange(R, 450-R):
        if wrf_landmask[(i-R):(i+R), (j-R):(j+R)].sum()>4:
            landmask_lake_fill[i,j] = 1

In [6]:
index_x = np.zeros((450,450))
index_y = np.zeros((450,450))

for i in np.arange(450):
    index_x[i,:] = np.ones(450)*i
    index_y[:,i] = np.ones(450)*i

In [7]:
ocean_xs = index_x[landmask_lake_fill==0]
ocean_ys = index_y[landmask_lake_fill==0]

In [8]:
dist_matrix = np.zeros((450,450))

for i in np.arange(450):
    #print(i)
    for j in np.arange(450):
        if landmask_lake_fill[i,j]==1:
            dist_matrix[i,j] = np.sqrt(((ocean_xs-i)**2 + (ocean_ys-j)**2).min())*6
        else:
            dist_matrix[i,j] = 9999

In [28]:
#outfile = rootdir + 'data/common_ref/SERDP6km.dist_to_coastal.nc'
            
#save_to_nc(outfile, dist_matrix)